In [1221]:
from __future__ import print_function
import csv
import pandas as pd
import numpy as np
import tensorflow as tf

In [1222]:
# Load data
train = pd.read_csv('../data/datosEntrenamiento.csv', header = None)
test = pd.read_csv('../data/datosPrueba.csv', header = None)
validation = pd.read_csv('../data/datosValidacion.csv', header = None)

In [1223]:
train=train.rename(columns = {36:'class'})
test=test.rename(columns = {36:'class'})

In [1224]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,class
0,76,94,98,76,72,85,90,72,68,85,...,62,71,79,85,62,67,75,85,62,2
1,68,89,86,72,68,85,90,76,68,94,...,67,71,75,81,62,67,75,85,71,2
2,92,115,115,94,92,111,120,91,84,106,...,92,93,107,109,92,93,111,113,92,1
3,84,106,111,87,84,98,111,87,84,98,...,92,93,107,113,87,93,107,113,87,1
4,88,111,115,91,88,111,120,87,88,111,...,87,88,107,109,87,88,107,109,87,1


In [1225]:
train_class_dummies = pd.get_dummies(list(train['class']))
train_class_dummies.columns = ['class1','class2','class3','class4','class5','class6']
test_class_dummies = pd.get_dummies(list(test['class']))
test_class_dummies.columns = ['class1','class2','class3','class4','class5','class6']

In [1226]:
train = pd.concat([train,train_class_dummies], axis = 1)
test = pd.concat([test,test_class_dummies], axis = 1)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,class,class1,class2,class3,class4,class5,class6
0,92,115,120,94,84,102,106,79,84,102,...,107,113,87,1,1,0,0,0,0,0
1,84,102,102,83,80,102,102,79,84,94,...,99,104,79,1,1,0,0,0,0,0
2,84,94,102,79,80,94,98,76,80,102,...,107,109,87,1,1,0,0,0,0,0
3,80,94,98,76,80,102,102,79,76,102,...,107,109,87,1,1,0,0,0,0,0
4,76,102,106,83,76,102,106,87,80,98,...,103,104,79,1,1,0,0,0,0,0


In [1227]:
x_test = test.drop(test.columns[list(range(36,43))], axis = 1).values
y_test = test.drop(test.columns[list(range(37))],axis = 1).values
x_train = train.drop(train.columns[list(range(36,43))], axis = 1).values
y_train = train.drop(test.columns[list(range(37))],axis = 1).values

In [1228]:
y_test.shape

(1287, 6)

In [1229]:
# Parameters
learning_rate = 0.1
num_steps = 300
batch_size = 100
display_step = num_steps / 10

In [1230]:
# Network Parameters
n_hidden_1 = 12
n_hidden_2 = 15
num_input = 36
num_classes = 6

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [1231]:
Y.shape

TensorShape([Dimension(None), Dimension(6)])

In [1232]:
X.shape

TensorShape([Dimension(None), Dimension(36)])

In [1233]:
y_train.shape

(4504, 6)

In [1234]:
x_train.shape

(4504, 36)

In [1235]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [1236]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [1237]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [1238]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [1240]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x = x_train
        batch_y = y_train
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test,
                                      Y: y_test}))

Step 1, Minibatch Loss= 6448.3423, Training Accuracy= 0.107
Step 30, Minibatch Loss= 321.8801, Training Accuracy= 0.526
Step 60, Minibatch Loss= 50.3633, Training Accuracy= 0.611
Step 90, Minibatch Loss= 25.6695, Training Accuracy= 0.663
Step 120, Minibatch Loss= 16.7565, Training Accuracy= 0.701
Step 150, Minibatch Loss= 11.7335, Training Accuracy= 0.713
Step 180, Minibatch Loss= 8.2205, Training Accuracy= 0.728
Step 210, Minibatch Loss= 7.8532, Training Accuracy= 0.620
Step 240, Minibatch Loss= 6.2677, Training Accuracy= 0.773
Step 270, Minibatch Loss= 4.9410, Training Accuracy= 0.780
Step 300, Minibatch Loss= 4.3342, Training Accuracy= 0.794
Optimization Finished!
Testing Accuracy: 0.8088578
